# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
#There are data collected from 17 marketing campaigns in this study.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [25]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [26]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [27]:
# Check for 'unknown' incidence in categorical columns
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan']

print("--- Percentage of 'unknown' values per column ---")
for col in categorical_cols:
    # count 'unknown' and divide by total length of dataframe
    unknown_pct = (df[col] == 'unknown').sum() / len(df) * 100
    print(f"{col}: {unknown_pct:.2f}%")

# Create a new binary column 'was_contacted' and replace 999 with '0' if they were not contacted or '1' if they were
# This fixes '999' value in pdays column. Additionally, whether a customer was contacted is likely a stronger predictor feature than the # of days
df['pdays_contacted'] = np.where(df['pdays'] == 999, 0, 1)

# Drop duration and pdays column
df = df.drop(columns=['duration', 'pdays'])

# Encode target column y: no = 0, yes = 1
df['y'] = df['y'].map({'no': 0, 'yes': 1})

--- Percentage of 'unknown' values per column ---
job: 0.80%
marital: 0.19%
education: 4.20%
default: 20.87%
housing: 2.40%
loan: 2.40%


In [28]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,pdays_contacted
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,0


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [7]:
#The primary business objective of analyzing this dataset is to optimize marketing efficiency by identifying the customers most likely to subscribe to a term deposit.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [29]:
# Encode categorical feature columns and scale numerical feature columns

categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 
                    'contact', 'month', 'day_of_week', 'poutcome']

numeric_cols = ['age', 'campaign', 'previous', 'emp.var.rate', 
                'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'pdays_contacted']

# One-hot encode categorical cols
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

#Scale numerical cols
scaler = StandardScaler()
df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])

# Create feature set (X) and target (y)
X = df_encoded.drop(columns=['y'])
y = df_encoded['y']

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [30]:
# Split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [31]:
# Calculate the baseline which should equal the % of the majority class in y_test
y_test_counts = y_test.value_counts()
majority_class_count = y_test_counts.max()
total_count = len(y_test)
baseline_acc = majority_class_count / total_count

In [32]:
print(baseline_acc)

0.8873458288821987


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
model_acc = accuracy_score(y_test, y_pred)

print(model_acc)

0.9018160629309507


### Problem 9: Score the Model

What is the accuracy of your model?

In [34]:
# The accuracy of the logistic regression model is 90.18% which is only slightly higher than the baseline (88.7%).

# Also checking the recall and precision scores:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")

# Generate confusion matrix
# [ [True Negatives,  False Positives],
#   [False Negatives, True Positives] ]
cm = confusion_matrix(y_test, y_pred)

print(cm)

# Based on precision/recall score and confusion matrix, we also can see that the model has a low recall and a high volume of false negatives.


Precision: 0.7041
Recall:    0.2216
[[9029  108]
 [ 903  257]]


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [23]:
# Create dictionary of models that we will test
models = {
    "Logistic Regression": lr,  # The one we trained earlier
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(), 
    "SVM": SVC()
}

results = []

for name, model in models.items():
    # Start timer
    start_time = time.time()
    
    # Fit the model
    model.fit(X_train, y_train)
    
    # End timer
    end_time = time.time()
    training_time = end_time - start_time
    
    # Calculate Train Accuracy (How well did it memorize the study material?)
    train_acc = model.score(X_train, y_train)
    
    # Calculate Test Accuracy (How well does it perform on the exam?)
    test_acc = model.score(X_test, y_test)
    
    # Store results
    results.append({
        "Model": name,
        "Training Time (s)": round(training_time, 4),
        "Train Accuracy": round(train_acc, 4),
        "Test Accuracy": round(test_acc, 4)
    })

results_df = pd.DataFrame(results)

# Summary table comparing each of 4 models and their training time, training accuracy and test accuracy
results_df

,Model,Training Time (s),Train Accuracy,Test Accuracy
0,Logistic Regression,0.3987,0.8998,0.9018
1,KNN,0.0175,0.9135,0.8937
2,Decision Tree,0.2518,0.9958,0.8468
3,SVM,97.0744,0.9052,0.9032


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [35]:
param_grids = {"Logistic Regression": {'C': [0.1, 1, 10],            # Inverse of regularization strength
                                       'solver': ['liblinear']       
    },
    "KNN": {'n_neighbors': [5, 15, 25],   
            'weights': ['uniform', 'distance']
    },
    "Decision Tree": {'max_depth': [5, 10, 15],     # Control tree height to prevent overfitting
                      'min_samples_split': [2, 10]  # Minimum samples required to split a node
    },
    "SVM": {'C': [0.1, 1, 10],
            'kernel': ['rbf'],            
            'gamma': ['scale']            
    }
}

# Define the models
models_to_tune = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(random_state=42)
}

# 2. TUNING LOOP
tuned_results = []

for name, model in models_to_tune.items():
       
    # Setup Grid Search
    # cv=3 splits data into 3 parts for validation (faster than default 5)
    grid_search = GridSearchCV(
        estimator=model, 
        param_grid=param_grids[name], 
        cv=3, 
        scoring='accuracy',
        n_jobs=-1  # Use all available CPU cores
    )
    
    # Run the search
    grid_search.fit(X_train, y_train)
    
    # Get the best model from the search
    best_model = grid_search.best_estimator_
    
    # 3. RE-EVALUATE THE BEST MODEL
    # We re-time the training of just the *best* model to get a fair "Training Time" comparison
    start_time = time.time()
    best_model.fit(X_train, y_train)
    end_time = time.time()
    train_time = end_time - start_time
    
    # Calculate Accuracies
    train_acc = best_model.score(X_train, y_train)
    test_acc = best_model.score(X_test, y_test)
    
    # Store results
    tuned_results.append({
        "Model": name,
        "Best Params": str(grid_search.best_params_),
        "Training Time (s)": round(train_time, 4),
        "Train Accuracy": round(train_acc, 4),
        "Test Accuracy": round(test_acc, 4)
    })

# 4. CREATE FINAL DATAFRAME
tuned_results_df = pd.DataFrame(tuned_results)
tuned_results_df['Overfitting Delta'] = tuned_results_df['Train Accuracy'] - tuned_results_df['Test Accuracy']
tuned_results_df = tuned_results_df.sort_values(by='Test Accuracy', ascending=False)

print("\n--- Tuned Model Comparison Results ---")
print(tuned_results_df)


--- Tuned Model Comparison Results ---
                 Model                                  Best Params  \
3                  SVM  {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}   
0  Logistic Regression              {'C': 1, 'solver': 'liblinear'}   
2        Decision Tree    {'max_depth': 5, 'min_samples_split': 10}   
1                  KNN    {'n_neighbors': 25, 'weights': 'uniform'}   

   Training Time (s)  Train Accuracy  Test Accuracy  Overfitting Delta  
3           124.5030          0.9052         0.9032             0.0020  
0             0.3320          0.8998         0.9020            -0.0022  
2             0.1720          0.9031         0.9020             0.0011  
1             0.0404          0.9029         0.8999             0.0030  


##### Questions